In [1]:
import pandas as pd
import requests
import json

In [2]:
with open('voting-points-formatted.json', 'rt') as f:
    vp = json.load(f)['stations']

In [43]:
def get_full_geo_data(session, station):
    return session.get(
        f'https://nominatim.openstreetmap.org/'
        f'reverse?format=geojson&lat={station["latitude"]}&lon={station["longitude"]}&accept-language=en'
    ).json()

In [44]:
session = requests.Session()

In [15]:
with open('geo-voting-points.json', 'rt') as f:
    full_data = json.load(f)
# full_data = []

In [209]:
for i, record in enumerate(vp[4000:]):
    print(4000 + i)
    full_data.append(get_full_geo_data(session, record))
    full_data[-1]['id'] = record['id']

4000
4001
4002
4003
4004
4005
4006
4007
4008
4009
4010
4011
4012
4013
4014
4015
4016
4017
4018
4019
4020
4021
4022
4023
4024
4025
4026
4027
4028
4029
4030
4031
4032
4033
4034
4035
4036
4037
4038
4039
4040
4041
4042
4043
4044
4045
4046
4047
4048
4049
4050
4051
4052
4053
4054
4055
4056
4057
4058
4059
4060
4061
4062
4063
4064
4065
4066
4067
4068
4069
4070
4071
4072
4073
4074
4075
4076
4077
4078
4079
4080
4081
4082
4083
4084
4085
4086
4087
4088
4089
4090
4091
4092
4093
4094
4095
4096
4097
4098
4099
4100
4101
4102
4103
4104
4105
4106
4107
4108
4109
4110
4111
4112
4113
4114
4115
4116
4117
4118
4119
4120
4121
4122
4123
4124
4125
4126
4127
4128
4129
4130
4131
4132
4133
4134
4135
4136
4137
4138
4139
4140
4141
4142
4143
4144
4145
4146
4147
4148
4149
4150
4151
4152
4153
4154
4155
4156
4157
4158
4159
4160
4161
4162
4163
4164
4165
4166
4167
4168
4169
4170
4171
4172
4173
4174
4175
4176
4177
4178
4179
4180
4181
4182
4183
4184
4185
4186
4187
4188
4189
4190
4191
4192
4193
4194
4195
4196
4197
4198
4199


5639
5640
5641
5642
5643
5644
5645
5646
5647
5648
5649
5650
5651
5652
5653
5654
5655
5656
5657
5658
5659
5660
5661
5662
5663
5664
5665
5666
5667
5668
5669
5670
5671
5672
5673
5674
5675
5676
5677
5678
5679
5680
5681
5682
5683
5684
5685
5686
5687
5688
5689
5690
5691
5692
5693
5694
5695
5696
5697
5698
5699
5700
5701
5702
5703
5704
5705
5706
5707
5708
5709
5710
5711
5712
5713
5714
5715
5716
5717
5718
5719
5720
5721
5722
5723
5724
5725
5726
5727
5728
5729
5730
5731
5732
5733
5734
5735
5736
5737
5738
5739
5740
5741
5742
5743
5744
5745
5746
5747
5748
5749
5750
5751
5752
5753
5754
5755
5756
5757
5758
5759
5760
5761
5762
5763
5764
5765
5766


In [222]:
with open('geo-voting-points.json', 'wt') as f:
    json.dump(full_data, f)

In [40]:
def create_clean_record(record):
    if len(record['features']) != 1:
        raise ValueError(f'Record {i} not parsed')
        
    
    address = record['features'][0]['properties']['address']
    
    if address['country_code'] != 'by':
        area = 'embassy'
        town = address['country_code']
        
    elif 'county' in address:
        town = address.get('town') or address.get('village') or address.get('hamlet') or address.get('city')
        
        if 'town' not in address or ('city' in address and 'village' not in address and 'hamlet' not in address):
            area = 'village'
        
        else:
            if address['county'].startswith(town):
                area = 'region-center'
            else:
                area = 'town'
            
    else:
        if 'city' in address:
            area = 'city'
            town = address['city']
        elif address.get('town') == 'Zhodzina':
            area = 'region-center'
            town = address['town']
        else:
            raise ValueError(address)
            
    town = town.strip()
    assert town
    
    return {
        'id': record['id'],
        'town': town,
        'area': area,
    }

In [48]:
clean_data = [create_clean_record(x) for x in full_data]    

In [49]:
clean_data[677]['town'] = 'Pinsk'
clean_data[955]['town'] = 'Vitebsk'
clean_data[1320]['town'] = 'Navapolack'
clean_data[1321]['town'] = 'Navapolack'

clean_data[1704]['town'] = 'Homel'
clean_data[1704]['area'] = 'city'

clean_data[2624]['area'] = 'village'

clean_data[3390]['town'] = 'Borisov'
clean_data[3391]['town'] = 'Borisov'

clean_data[3999]['area'] = 'village'
clean_data[3999]['town'] = 'Яловица'

clean_data[4232]['area'] = 'village'
clean_data[4411]['area'] = 'village'

clean_data[5388]['town'] = 'Minsk'
clean_data[5388]['area'] = 'city'
clean_data[5389]['town'] = 'Minsk'
clean_data[5389]['area'] = 'city'

In [50]:
def fix_area(record):
    if record['town'] in (
        'Baranavičy', 
        'Biaroza', 
        'Belaazersk', 
        'Ivacevichi', 
        'Kamyanyets', 
        'Luniniec', 
        'Mikashevichy', 
        'Pinsk',
        'Lyepyel',
        'Polack',
        'Miory',
        'Sianno',
        'Sharkaushchyna',
        'Orsha',
        'Vetka',
        'Zhytkavichy',
        'Zhlobin',
        'Lelchytsy',
        'Mazyr',
        'Narowlya',
        'Aktsyabarski',
        'Rahachow',
        'Svietlahоrsk',
        'Chojniki',
        'Dziatlava',
        'Zeĺva',
        'Lida',
        'Skidzieĺ',
        'Berezino',
        'Borisov',
        'Kapyĺ',
        'Maladzeĉna',
        'Maryina Horka',
        'Sluck',
        'Soligorsk',
        'Babruysk',
        'Bykhaw',
        'Krasnapollye',
        'Mstislavl',
        'Shklov',
    ):
        record['area'] = 'region-center'
        
    if record['town'] in ('Navapolack', ):
        record['area'] = 'town'

In [51]:
for record in clean_data:
    fix_area(record)

In [52]:
for i in range(len(clean_data)):
    clean_data[i]['latitude'] = vp[i]['latitude']
    clean_data[i]['longitude'] = vp[i]['longitude']

In [53]:
pd.DataFrame(clean_data).to_csv('geo-by-area.csv', index=False)

In [36]:
clean_data[1320]['town']

'Боровухский сельский Совет'

In [37]:
full_data[1320]

{'type': 'FeatureCollection',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'features': [{'type': 'Feature',
   'properties': {'place_id': 236173841,
    'osm_type': 'relation',
    'osm_id': 8022794,
    'place_rank': 30,
    'category': 'amenity',
    'type': 'school',
    'importance': 0,
    'addresstype': 'amenity',
    'name': 'СШ № 15',
    'display_name': 'СШ № 15, Базарная улица, Baravucha, Боровухский сельский Совет, Navapolack, Vitsebsk Region, 211502, Belarus',
    'address': {'amenity': 'СШ № 15',
     'road': 'Базарная улица',
     'suburb': 'Baravucha',
     'city': 'Боровухский сельский Совет',
     'state': 'Vitsebsk Region',
     'postcode': '211502',
     'country': 'Belarus',
     'country_code': 'by'}},
   'bbox': [28.596251, 55.5867752, 28.5997488, 55.5888748],
   'geometry': {'type': 'Point',
    'coordinates': [28.598128946153857, 55.587823650000004]}}],
 'id': '02-074-0034'}

In [39]:
vp[1321]

{'id': '02-074-0035',
 'name': 'Участок для голосования № 35',
 'description': 'г.Новополоцк (бывший г.п.Боровуха), ул. Базарная, д.7а, государственное учреждение образования «Средняя школа № 15 г.Новополоцка».',
 'longitude': 28.597786,
 'latitude': 55.587433,
 'applied': 1}